In [1]:
import pandas
reviews = pandas.read_csv('./data/reviews10000.csv', usecols=["AuthorId", "RecipeId", "Rating"])
reviews

,RecipeId,AuthorId,Rating
0,992,2008,5
1,44,2085,5
2,148,2156,0
3,517,2046,5
4,278,1773,4
...,...,...,...
8187,1356,2002736430,5
8188,1356,2002851165,5
8189,1035,2002867543,5
8190,607,2002874060,5


In [2]:
recipes = pandas.read_csv('./data/recipes10000.csv')

In [ ]:
import pandas as pd

from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate


# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(reviews[["AuthorId", "RecipeId", "Rating"]], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

In [ ]:
from surprise import Dataset, KNNBasic

# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

In [ ]:
data.raw_ratings

In [ ]:
#  Prediction(uid=1634, iid=16193, r_ui=5.0, est=3.629641121299654, details={'was_impossible': False}),

uid = 1634  # raw user id (as in the ratings file). They are **strings**!
iid = 16193  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)

In [ ]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

In [ ]:
predictions

# User recom

In [ ]:
liked = [648]
recipes[recipes['RecipeId'].isin(liked)]

In [ ]:
from surprise import Dataset, KNNBasic

new = pd.DataFrame([[0, i, 5] for i in liked],
                   columns=["AuthorId", "RecipeId", "Rating"])
new_reviews = pd.concat([reviews, new])

new_data = Dataset.load_from_df(new_reviews[["AuthorId", "RecipeId", "Rating"]], reader)
new_trainset = new_data.build_full_trainset()

# Build an algorithm, and train it.
new_algo = KNNBasic()
new_algo.fit(new_trainset)


uid = 0  # raw user id (as in the ratings file). They are **strings**!
iid = 4684  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = [new_algo.predict(uid, iid) for iid in new_reviews['RecipeId']]
top_iid = sorted(set([(i.iid, i.est) for i in pred if not i.details['was_impossible']]), key=lambda i: i[1])
print(top_iid)
top_iid = [i[0] for i in top_iid if i[1] > 4.0]
top_iid

In [ ]:
recipes[recipes['RecipeId'].isin(top_iid)]

## Func

In [3]:
def recom(recipes, reviews, liked, disliked):
    import pandas as pd
    from surprise import Dataset, NormalPredictor, Reader, KNNBasic
    from surprise.model_selection import cross_validate

    new = pd.DataFrame([[0, i, 5] for i in liked] + [[0, i, 0] for i in disliked],
                       columns=["AuthorId", "RecipeId", "Rating"])
    new_reviews = pd.concat([reviews, new])

    reader = Reader(rating_scale=(0, 5))
    new_data = Dataset.load_from_df(new_reviews[["AuthorId", "RecipeId", "Rating"]], reader)
    new_trainset = new_data.build_full_trainset()

    # Build an algorithm, and train it.
    new_algo = KNNBasic()
    new_algo.fit(new_trainset)


    uid = 0  # id that's guaranteed to not exist

    # get a prediction for specific users and items.
    pred = [new_algo.predict(uid, iid) for iid in new_reviews['RecipeId']]
    top_iid = sorted(set([(i.iid, i.est) for i in pred if not i.details['was_impossible']]), key=lambda i: i[1])[-100:]
    top_iid = [i[0] for i in top_iid if i[1] > 4.0 and i[0] not in liked + disliked]
    return top_iid

In [5]:
liked = [648]
disliked = [647]
recipes[recipes['RecipeId'].isin(liked)]

,Unnamed: 0,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
560,560,648,Cucumber Salad with Spicy Wasabi Dressing,1533,Dancer,PT5M,PT25M,PT30M,1999-08-14T06:12:00Z,Make and share this Cucumber Salad with Spicy ...,...,0.4,0.0,4.0,8.6,1.2,4.7,1.5,4.0,NaN,"c(""Mix the wasabi with 1/2 teaspoon warm water..."


In [6]:
recipes[recipes['RecipeId'].isin(disliked)]

,Unnamed: 0,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
559,559,647,Awesome Apple Bread,1535,Marg CaymanDesigns,PT2H50M,PT20M,PT3H10M,1999-09-13T14:32:00Z,This bread is so moist and flavorful that my k...,...,16.8,279.0,1906.5,370.8,10.3,220.2,29.3,NaN,2-3 loaves,"c(""Combine and set aside the oil, eggs, sugar,..."


In [7]:
recoms = recom(recipes, reviews, liked, disliked)
recipes[recipes['RecipeId'].isin(recoms)]

Computing the msd similarity matrix...
Done computing similarity matrix.


,Unnamed: 0,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
11,11,49,Chicken Breasts Lombardi,174711,Queen Dragon Mom,PT30M,PT45M,PT1H15M,1999-08-14T19:58:00Z,Make and share this Chicken Breasts Lombardi r...,...,13.0,203.0,848.9,13.7,0.6,2.0,57.9,6.0,NaN,"c(""Cook mushrooms in 2 tbsp butter in a large ..."
18,18,56,Buttermilk Pie,1581,thefensk,PT1H,PT20M,PT1H20M,1999-08-30T10:02:00Z,This recipe was originally noted by my wife on...,...,9.8,101.5,269.8,52.2,0.7,39.9,5.0,8.0,NaN,"c(""Preheat oven to 400°F."", ""Beat the butter a..."
37,37,76,Alfredo Sauce,1535,Marg CaymanDesigns,PT10M,PT5M,PT15M,1999-09-15T04:26:00Z,This is my son's favorite meal. I make it with...,...,31.5,159.0,312.4,2.5,0.0,0.2,8.7,4.0,NaN,"c(""Place butter in microwave safe pot and heat..."
53,53,93,Blue Stuffed Mushrooms,1533,Dancer,NaN,PT15M,PT15M,1999-08-12T06:42:00Z,Make and share this Blue Stuffed Mushrooms rec...,...,12.1,53.8,482.8,3.5,0.9,1.0,9.9,4.0,NaN,"c(""In a medium bowl, combine the blue cheese a..."
85,85,129,Champagne Punch,1592,polar-express,NaN,PT5M,PT5M,1999-08-21T11:22:00Z,Here is a good punch recipe for any special oc...,...,0.0,0.0,14.2,41.6,0.2,38.1,0.6,20.0,NaN,"c(""Mix the juice concentrates in punch bowl (d..."
108,108,155,Light Cucumber Soup,1530,Carole Reu,PT8H,PT15M,PT8H15M,1999-09-07T16:24:00Z,Make and share this Light Cucumber Soup recipe...,...,0.8,7.2,80.0,20.3,1.1,9.2,6.2,4.0,NaN,"c(""Cut cucumbers into large chunks; place in a..."
149,149,198,Chinese Hot and Sour Pork Soup,1545,Nancy Van Ess,NaN,PT40M,PT40M,1999-09-18T11:13:00Z,Make and share this Chinese Hot and Sour Pork ...,...,2.9,31.1,1459.3,32.1,3.0,10.5,31.6,4.0,NaN,"c(""Bring stock to a simmer."", ""Add soy sauce, ..."
161,161,211,Christmas Cranberry Salad,1572,Ed Paulhus,PT5M,PT8H,PT8H5M,1999-09-13T03:04:00Z,Make and share this Christmas Cranberry Salad ...,...,7.3,40.8,100.3,44.9,2.5,35.5,2.3,8.0,NaN,"c(""Cover marshmallows and cranberries with sug..."
170,170,220,Chocolate Zucchini Bread,1604,itsjustme,PT1H,PT25M,PT1H25M,1999-08-07T09:18:00Z,Make and share this Chocolate Zucchini Bread r...,...,3.7,55.8,392.8,67.6,2.7,40.9,6.3,10.0,2 loaves,"c(""In a mixing bowl, beat eggs, oil, sugar and..."
178,178,228,Chocolate Coffee,1530,Carole Reu,NaN,PT10M,PT10M,1999-08-21T11:14:00Z,Make and share this Chocolate Coffee recipe fr...,...,11.9,29.9,43.5,20.3,3.1,11.5,5.0,6.0,NaN,"c(""Bring chocolate and half and half to a boil..."
